Misc imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import matplotlib.pyplot as plt
from six.moves.cPickle import load
import cPickle
import numpy as np

from snorkel import SnorkelSession
from snorkel.parser import XMLMultiDocPreprocessor, CorpusParser
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser.corenlp import StanfordCoreNLPServer
from snorkel.models import Document, Sentence, Candidate, candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.viewer import SentenceNgramViewer
from snorkel.annotations import LabelAnnotator, load_gold_labels, FeatureAnnotator, save_marginals, load_marginals
from snorkel.learning import SparseLogisticRegression, GenerativeModel, RandomSearch
from snorkel.learning.structure import DependencySelector
from snorkel.learning.utils import MentionScorer
# from snorkel.contrib.rnn import reRNN

import matchers
import LF
from candidate_adjective_fixer import *
from load_external_annotations_new import load_external_labels

session = SnorkelSession()

BiomarkerMedium = candidate_subclass('BiomarkerMedium', ['biomarker', 'medium'])



/Users/gmachiraju/py2_kernel/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /usr/local/Cellar/python/2.7.9/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Helper functions

In [2]:
#------------------
# Helper Functions
#------------------

def graBMandidates(extractor, schema):
    # Candidate Counts
    for k, sents in enumerate([train_sents, dev_sents, test_sents]):
        extractor.apply(sents, split=k, clear=False)
        print "Number of candidates: ", session.query(schema).filter(schema.split == k).count()
        session.commit()
        
    train_cands = session.query(schema).filter(
        schema.split == 0).all()
    dev_cands = session.query(schema).filter(
        schema.split == 1).all()
    test_cands = session.query(schema).filter(
        schema.split == 2).all()

    return [train_cands, dev_cands, test_cands]

In [3]:
#-----------------------
# Setup & Preprocessing
#-----------------------

# Instantiate the Session
session = SnorkelSession()

# Doc Preprocessing
file_path = 'articles/training.xml'
train_preprocessor = XMLMultiDocPreprocessor(
    path=file_path,
    doc='.//article',
    text='.//front/article-meta/abstract/p/text()',
    id='.//front/article-meta/article-id/text()'
)

file_path = 'articles/development.xml'
dev_preprocessor = XMLMultiDocPreprocessor(
    path=file_path,
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()'
)

file_path = 'articles/testcorpus.xml'
test_preprocessor = XMLMultiDocPreprocessor(
    path=file_path,
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()'
)

# Parsing
# corenlp_server = StanfordCoreNLPServer(version="3.6.0", num_threads=4, port=12348)
# corpus_parser = CorpusParser(corenlp_server, parser=Spacy())
corpus_parser = CorpusParser(parser=Spacy())
# corpus_parser = CorpusParser()

# Note: Parallelism can be run with a Postgres DBMS, but not SQLite
corpus_parser.apply(list(train_preprocessor))
corpus_parser.apply(list(dev_preprocessor), clear=False)
corpus_parser.apply(list(test_preprocessor), clear=False)





Clearing existing...
Running UDF...
[========================================] 100%

Running UDF...
[========================================] 100%

Running UDF...
[========================================] 100%



In [4]:
# Retrieving Stable IDs for each of the candidate sentences
with open('articles/doc_ids.pkl', 'rb') as f:
    train_ids, dev_ids, test_ids = load(f)

train_ids, dev_ids, test_ids = set(train_ids), set(dev_ids), set(test_ids)
train_sents, dev_sents, test_sents = set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()


# Assigning each sentence to {train,dev,test}-set based on Stable ID
for i, doc in enumerate(docs):
    for s in doc.sentences:
        if doc.name in train_ids:
            train_sents.add(s)
        elif doc.name in dev_ids:
            dev_sents.add(s)
        elif doc.name in test_ids:
            test_sents.add(s)
        else:
            raise Exception(
                'ID <{0}> not found in any id set'.format(doc.name))

In [5]:
#----------------------
# Candidate Extraction
#----------------------

# Defining the Candidate Schemas
# BiomarkerCondition = candidate_suBMlass('BiomarkerCondition', ['biomarker', 'condition'])

# BiomarkerDrug = candidate_suBMlass('BiomarkerDrug', ['biomarker', 'drug'])
# BiomarkerMedium = candidate_suBMlass('BiomarkerMedium', ['biomarker', 'medium'])
# BiomarkerType = candidate_suBMlass('BiomarkerType', ['biomarker', 'typ3'])
# # BiomarkerLevelUnit = candidate_suBMlass('BiomarkerLevelUnit', ['biomarker', 'level', 'unit'])
#can eventually add MEASUREMENT and COHORT SIZE among other entities

# N-grams: the probabilistic search space of our entities
biomarker_ngrams = Ngrams(n_max=1)
# condition_ngrams = Ngrams(n_max=7)
# drug_ngrams = Ngrams(n_max=5)
medium_ngrams = Ngrams(n_max=5)
# type_ngrams = Ngrams(n_max=5)  # <--- Q: should we cut these down?
# # level_ngrams = Ngrams(n_max=1)
# unit_ngrams = Ngrams(n_max=1)

# Construct our Matchers
bMatcher = matchers.getBiomarkerMatcher()
# cMatcher = matchers.getConditionMatcher()
# dMatcher = matchers.getDrugMatcher()
mMatcher = matchers.getMediumMatcher()
# tMatcher = matchers.getTypeMatcher()
# lMatcher = matchers.getLevelMatcher()
# uMatcher = matchers.getUnitMatcher()

# Building the CandidateExtractors
# candidate_extractor_BM = CandidateExtractor(BiomarkerCondition, [biomarker_ngrams, condition_ngrams], [bMatcher, cMatcher])
# candidate_extractor_BD = CandidateExtractor(BiomarkerDrug, [biomarker_ngrams, drug_ngrams], [bMatcher, dMatcher])
candidate_extractor_BM = CandidateExtractor(BiomarkerMedium, [biomarker_ngrams, medium_ngrams], [bMatcher, mMatcher])
# candidate_extractor_BT = CandidateExtractor(BiomarkerType, [biomarker_ngrams, type_ngrams], [bMatcher, tMatcher])
# candidate_extractor_BLU = CandidateExtractor(BiomarkerLevelUnit, [biomarker_ngrams, level_ngrams, unit_ngrams], [bMatcher, lMatcher, uMatcher])

# List of Candidate Sets for each relation type: [train, dev, test]
# cands_BM = graBMandidates(candidate_extractor_BM, BiomarkerCondition)
# cands_BD = graBMandidates(candidate_extractor_BD, BiomarkerDrug)
cands_BM = graBMandidates(candidate_extractor_BM, BiomarkerMedium)
# cands_BT = graBMandidates(candidate_extractor_BT, BiomarkerType)
# cands_BLU = graBMandidates(candidate_extractor_BLU, BiomarkerLevelUnit)





Running UDF...
[========================================] 100%

Number of candidates:  781
Running UDF...
[========================================] 100%

Number of candidates:  396
Running UDF...
[========================================] 100%

Number of candidates:  48


In [6]:
session.rollback()
print "Number of dev BM candidates without adj. boosting: ", len(cands_BM[1])
add_adj_candidate_BM(session, BiomarkerMedium, cands_BM[1], 0)
# fix_specificity(session, BiomarkerCondition, cands_BM[1])
print "Number of dev BM candidates with adj. boosting: ", session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 1).count()
session.commit()

Number of dev BM candidates without adj. boosting:  396
 added to blood
 added to plasma
 added to blood
 added to plasma
 added to blood
 added to plasma
maternal  added to serum
Checking Biomarker: 28262321::span:1767:1769
Checking Condition: 28262321::span:129:142
0
Couldn't find: 28262321::span:129:142
ADDING NEW CANDIDATE
cord  added to serum
Checking Biomarker: 28262321::span:1767:1769
Checking Condition: 28262321::span:233:242
0
Couldn't find: 28262321::span:233:242
ADDING NEW CANDIDATE
maternal  added to serum
Checking Biomarker: 28262321::span:1708:1710
Checking Condition: 28262321::span:129:142
1
cord  added to serum
Checking Biomarker: 28262321::span:1708:1710
Checking Condition: 28262321::span:233:242
1
 added to plasma
 added to serum
 added to serum
 added to serum
 added to serum
 added to serum
 added to CSF
 added to CSF
 added to CSF
tumor  added to blood
Checking Biomarker: 28264009::span:273:275
Checking Condition: 28264009::span:72:82
0
Couldn't find: 28264009::spa

ADDING NEW CANDIDATE
 added to serum
 added to serum
 added to serum
 added to serum
 added to serum
 added to serum
 added to serum
 added to serum
 added to serum
 added to astrocytes
 added to CSF
 added to CSF
 added to CSF
 added to CSF
 added to CSF
 added to CSF
 added to CSF
 added to CSF
 added to CSF
 added to Plasma
 added to serum
 added to Serum
 added to serum
 added to Serum
 added to serum
 added to Serum
 added to serum
 added to Serum
 added to lymph
 added to plasma
 added to plasma
 added to plasma
treatment  added to plasma
Checking Biomarker: 28261341::span:305:307
Checking Condition: 28261341::span:107:122
0
Couldn't find: 28261341::span:107:122
ADDING NEW CANDIDATE
 added to Blood
 added to Blood
 added to plasma
 added to serum
 added to serum
 added to serum
procedural  added to serum
Checking Biomarker: 28260767::span:932:932
Checking Condition: 28260767::span:156:171
0
Couldn't find: 28260767::span:156:171
ADDING NEW CANDIDATE
procedural  added to serum
Chec

In [8]:

dev_cands   = session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 1).order_by(BiomarkerMedium.id).all()



from LF import *
labeled = []
for c in dev_cands:
    if(LF_distance_far(c) != 0):
        labeled.append(c)
print len(labeled)


LFs_BM = [LF_distance_far, LF_colon, LF_known_abs, LF_single_letter, LF_roman_numeral, LF_common_2000, LF_same_thing]

220


In [9]:
from snorkel.annotations import LabelAnnotator
BM_labeler = LabelAnnotator(lfs=LFs_BM)

In [10]:


np.random.seed(1701)
%time L_train_BM = BM_labeler.apply(split=0)
L_train_BM

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 5.14 s, sys: 93.8 ms, total: 5.23 s
Wall time: 5.21 s


<814x7 sparse matrix of type '<type 'numpy.int64'>'
	with 1272 stored elements in Compressed Sparse Row format>

IndexError: index out of bounds

In [ ]:
%time L_train_BM = BM_labeler.load_matrix(session, split=0)
L_train_BM

In [ ]:
L_train_BM.get_candidate(session, 0)

In [ ]:
L_train_BM.get_key(session, 0)

In [ ]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
gen_model.train(L_train_BM, epochs=100, decay=0.95, step_size=0.1 / L_train_BM.shape[0], reg_param=1e-6)

In [ ]:
gen_model.weights.lf_accuracy

In [ ]:
train_marginals = gen_model.marginals(L_train_BM)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(train_marginals, bins=20)
plt.show()

In [ ]:
L_dev = BM_labeler.apply_existing(split=1)

In [ ]:
from snorkel.annotations import save_marginals
%time save_marginals(session, L_train_BM, train_marginals)

In [ ]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

In [13]:
train_cands = session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 0).order_by(BiomarkerMedium.id).all()
dev_cands   = session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 1).order_by(BiomarkerMedium.id).all()
test_cands  = session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 1).order_by(BiomarkerMedium.id).all()

In [14]:
from snorkel.annotations import load_gold_labels
load_external_labels(session, BiomarkerMedium, 'Biomarker', 'Medium', 'articles/medium_gold_labels.tsv', dev_cands, annotator_name='gold')
load_external_labels(session, BiomarkerMedium, 'Biomarker', 'Medium', 'articles/medium_gold_labels.tsv', test_cands, annotator_name='gold')

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=1)




AnnotatorLabels created: 0
AnnotatorLabels created: 0
AnnotatorLabels created: 0
AnnotatorLabels created: 0


In [20]:
print len(train_cands)
print len(dev_cands)
from snorkel.lf_helpers import test_LF
tp, fp, tn, fn = test_LF(session, LF_known_abs, split=1, annotator_name='gold')

814
396


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        50,
    'n_epochs':   30,
    'dropout':    0.25,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

In [ ]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))


In [ ]:
print len(test_cands)

In [ ]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

In [ ]:
lstm.save_marginals(session, test_cands)

In [ ]:
predictions = lstm.predictions(train_cands)

In [ ]:
i = 0
for prediction in predictions: 
    if(prediction == 1):
        i+=1
print i

In [ ]:
i = 0
while( i< len(train_cands)):
    print("Candidate: {}. Prediction: {}").format(train_cands[i], predictions[i])
    i += 1

In [ ]:
lstm.save("mediums")